In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, current_date, when,datediff,count,avg

# Create a Spark session
spark = SparkSession.builder.appName("EmployeeStatus").getOrCreate()

# Employee data
employees = [
    ("karthik", "2024-11-01"),
    ("neha", "2024-10-20"),
    ("priya", "2024-10-28"),
    ("mohan", "2024-11-02"),
    ("ajay", "2024-09-15"),
    ("vijay", "2024-10-30"),
    ("veer", "2024-10-25"),
    ("aatish", "2024-10-10"),
    ("animesh", "2024-10-15"),
    ("nishad", "2024-11-01"),
    ("varun", "2024-10-05"),
    ("aadil", "2024-09-30")
]

# Create DataFrame
employees_df = spark.createDataFrame(employees, ["name", "last_checkin"])
employees_df = employees_df.withColumn("last_checkin", col("last_checkin").cast("date")) \
    .withColumn("name", expr("initcap(name)")) \
    .withColumn("status", when(datediff(current_date(), col("last_checkin")) <= 7, "Active").otherwise("Inactive"))


# Show the result
employees_df.show()


+-------+------------+--------+
|   name|last_checkin|  status|
+-------+------------+--------+
|Karthik|  2024-11-01|  Active|
|   Neha|  2024-10-20|Inactive|
|  Priya|  2024-10-28|  Active|
|  Mohan|  2024-11-02|  Active|
|   Ajay|  2024-09-15|Inactive|
|  Vijay|  2024-10-30|  Active|
|   Veer|  2024-10-25|Inactive|
| Aatish|  2024-10-10|Inactive|
|Animesh|  2024-10-15|Inactive|
| Nishad|  2024-11-01|  Active|
|  Varun|  2024-10-05|Inactive|
|  Aadil|  2024-09-30|Inactive|
+-------+------------+--------+



In [0]:

sales = [
("karthik", 60000),
("neha", 48000),
("priya", 30000),
("mohan", 24000),
("ajay", 52000),
("vijay", 45000),
("veer", 70000),
("aatish", 23000),
("animesh", 15000),
("nishad", 8000),
("varun", 29000),
("aadil", 32000)
]
sales_df = spark.createDataFrame(sales, ["name", "total_sales"])
# Capitalize name, determine performance status, and aggregate total sales by performance status
sales_df = sales_df.withColumn("name", expr("initcap(name)")) \
    .withColumn("performance_status", when(col("total_sales") > 50000, "Excellent")
                .when((col("total_sales") > 25000) & (col("total_sales") <= 50000), "Good")
                .otherwise("Needs Improvement"))
    
sales_df.show()
    
# Aggregate total sales by performance status
aggregated_df = sales_df.groupBy("performance_status").sum("total_sales").withColumnRenamed("sum(total_sales)", "total_sales")

# Show the result
aggregated_df.show()


+-------+-----------+------------------+
|   name|total_sales|performance_status|
+-------+-----------+------------------+
|Karthik|      60000|         Excellent|
|   Neha|      48000|              Good|
|  Priya|      30000|              Good|
|  Mohan|      24000| Needs Improvement|
|   Ajay|      52000|         Excellent|
|  Vijay|      45000|              Good|
|   Veer|      70000|         Excellent|
| Aatish|      23000| Needs Improvement|
|Animesh|      15000| Needs Improvement|
| Nishad|       8000| Needs Improvement|
|  Varun|      29000|              Good|
|  Aadil|      32000|              Good|
+-------+-----------+------------------+

+------------------+-----------+
|performance_status|total_sales|
+------------------+-----------+
|         Excellent|     182000|
|              Good|     184000|
| Needs Improvement|      70000|
+------------------+-----------+



In [0]:
workload = [
("karthik", "ProjectA", 120),
("karthik", "ProjectB", 100),
("neha", "ProjectC", 80),
("neha", "ProjectD", 30),
("priya", "ProjectE", 110),
("mohan", "ProjectF", 40),
("ajay", "ProjectG", 70),
("vijay", "ProjectH", 150),
("veer", "ProjectI", 190),
("aatish", "ProjectJ", 60),
("animesh", "ProjectK", 95),
("nishad", "ProjectL", 210),
("varun", "ProjectM", 50),
("aadil", "ProjectN", 90)
]
workload_df = spark.createDataFrame(workload, ["name", "project", "hours"])
workload_df = workload_df.withColumn("name" ,expr("initcap(name)"))\
               .withColumn("WorkLoad_level",when(col("hours")>200,"Overloaded")
               .when((col("hours") > 100) & (col("hours") <= 200), "Balanced")
               .otherwise("Underutilized"))
workload_df_agg = workload_df.groupby("WorkLoad_level").sum("hours").withColumnRenamed("sum(hours)", "total_hours")
workload_df.show()
workload_df_agg.show()

+-------+--------+-----+--------------+
|   name| project|hours|WorkLoad_level|
+-------+--------+-----+--------------+
|Karthik|ProjectA|  120|      Balanced|
|Karthik|ProjectB|  100| Underutilized|
|   Neha|ProjectC|   80| Underutilized|
|   Neha|ProjectD|   30| Underutilized|
|  Priya|ProjectE|  110|      Balanced|
|  Mohan|ProjectF|   40| Underutilized|
|   Ajay|ProjectG|   70| Underutilized|
|  Vijay|ProjectH|  150|      Balanced|
|   Veer|ProjectI|  190|      Balanced|
| Aatish|ProjectJ|   60| Underutilized|
|Animesh|ProjectK|   95| Underutilized|
| Nishad|ProjectL|  210|    Overloaded|
|  Varun|ProjectM|   50| Underutilized|
|  Aadil|ProjectN|   90| Underutilized|
+-------+--------+-----+--------------+

+--------------+-----------+
|WorkLoad_level|total_hours|
+--------------+-----------+
|      Balanced|        570|
| Underutilized|        615|
|    Overloaded|        210|
+--------------+-----------+



In [0]:
"""Determine whether an employee has "Excessive Overtime" if their weekly hours exceed 60,
"Standard Overtime" if between 45-60 hours, and "No Overtime" if below 45 hours. Capitalize each
name and group by overtime status."""


employees = [
("karthik", 62),
("neha", 50),
("priya", 30),
("mohan", 65),
("ajay", 40),
("vijay", 47),
("veer", 55),
("aatish", 30),
("animesh", 75),
("nishad", 60)
]
employees_df = spark.createDataFrame(employees, ["name", "hours_worked"])
# Apply overtime classification and capitalize names
employees_df = employees_df.withColumn(
    "overtime_status",
    when(col("hours_worked") > 60, "Excessive Overtime")
    .when((col("hours_worked") >= 45) & (col("hours_worked") <= 60), "Standard Overtime")
    .otherwise("No Overtime")
).withColumn("name", expr("initcap(name)"))

# Group by overtime status and show results
employees_df.groupBy("overtime_status").count().show()

employees_df.show()

+------------------+-----+
|   overtime_status|count|
+------------------+-----+
|Excessive Overtime|    3|
| Standard Overtime|    4|
|       No Overtime|    3|
+------------------+-----+

+-------+------------+------------------+
|   name|hours_worked|   overtime_status|
+-------+------------+------------------+
|Karthik|          62|Excessive Overtime|
|   Neha|          50| Standard Overtime|
|  Priya|          30|       No Overtime|
|  Mohan|          65|Excessive Overtime|
|   Ajay|          40|       No Overtime|
|  Vijay|          47| Standard Overtime|
|   Veer|          55| Standard Overtime|
| Aatish|          30|       No Overtime|
|Animesh|          75|Excessive Overtime|
| Nishad|          60| Standard Overtime|
+-------+------------+------------------+



In [0]:

"""Classify each vehicle’s mileage as "High Efficiency" if mileage is above 25 MPG, "Moderate Efficiency"
if between 15-25 MPG, and "Low Efficiency" if below 15 MPG."""

vehicles = [
("CarA", 30),
("CarB", 22),
("CarC", 18),
("CarD", 15),
("CarE", 10),
("CarF", 28),
("CarG", 12),
("CarH", 35),
("CarI", 25),
("CarJ", 16)
]
vehicles_df = spark.createDataFrame(vehicles, ["vehicle_name", "mileage"])
vehicles_df = vehicles_df.withColumn("Efficiency",when(col("mileage")>25,"High Efficiency")
                                     .when((col("mileage") >= 15) & (col("mileage") <= 25), "Moderate Efficiency")
                                     .otherwise("Low Efficiency")
                                     )
vehicles_df.show()

+------------+-------+-------------------+
|vehicle_name|mileage|         Efficiency|
+------------+-------+-------------------+
|        CarA|     30|    High Efficiency|
|        CarB|     22|Moderate Efficiency|
|        CarC|     18|Moderate Efficiency|
|        CarD|     15|Moderate Efficiency|
|        CarE|     10|     Low Efficiency|
|        CarF|     28|    High Efficiency|
|        CarG|     12|     Low Efficiency|
|        CarH|     35|    High Efficiency|
|        CarI|     25|Moderate Efficiency|
|        CarJ|     16|Moderate Efficiency|
+------------+-------+-------------------+



In [0]:

"""Classify students based on their scores as "Excellent" if score is 90 or above, "Good" if between 75-
89, and "Needs Improvement" if below 75. Count students in each category."""

students = [
("karthik", 95),
("neha", 82),
("priya", 74),
("mohan", 91),
("ajay", 67),
("vijay", 80),
("veer", 85),
("aatish", 72),
("animesh", 90),
("nishad", 60)
]
students_df = spark.createDataFrame(students, ["name", "score"])
students_df = students_df.withColumn("Classify",when(col("score")>=90,"Excellent")
                         .when((col("score")>=75) & (col("score") <= 89) ,"GOOD")
                         .otherwise("Needs Improvement"))\
                        .withColumn("name",expr("initcap(name)"))
students_df.show()

+-------+-----+-----------------+
|   name|score|         Classify|
+-------+-----+-----------------+
|Karthik|   95|        Excellent|
|   Neha|   82|             GOOD|
|  Priya|   74|Needs Improvement|
|  Mohan|   91|        Excellent|
|   Ajay|   67|Needs Improvement|
|  Vijay|   80|             GOOD|
|   Veer|   85|             GOOD|
| Aatish|   72|Needs Improvement|
|Animesh|   90|        Excellent|
| Nishad|   60|Needs Improvement|
+-------+-----+-----------------+



In [0]:
"""Classify inventory stock levels as "Overstocked" if stock exceeds 100, "Normal" if between 50-100,
and "Low Stock" if below 50. Aggregate total stock in each category."""


inventory = [
("ProductA", 120),
("ProductB", 95),
("ProductC", 45),
("ProductD", 200),
("ProductE", 75),
("ProductF", 30),
("ProductG", 85),
("ProductH", 100),
("ProductI", 60),
("ProductJ", 20)
]
inventory_df = spark.createDataFrame(inventory, ["product_name", "stock_quantity"])
inventory_df = inventory_df.withColumn("Classify",when(col("stock_quantity")>100,"Overstocked")
                            .when((col("stock_quantity")>=50) & (col("stock_quantity")<=100),"Normal")
                            .otherwise("Low Stock"))
inventory_df_agg = inventory_df.groupby("Classify").sum("stock_quantity").withColumnRenamed("sum(stock_quantity)","Total Quantity")
inventory_df.show()
inventory_df_agg.show()


+------------+--------------+-----------+
|product_name|stock_quantity|   Classify|
+------------+--------------+-----------+
|    ProductA|           120|Overstocked|
|    ProductB|            95|     Normal|
|    ProductC|            45|  Low Stock|
|    ProductD|           200|Overstocked|
|    ProductE|            75|     Normal|
|    ProductF|            30|  Low Stock|
|    ProductG|            85|     Normal|
|    ProductH|           100|     Normal|
|    ProductI|            60|     Normal|
|    ProductJ|            20|  Low Stock|
+------------+--------------+-----------+

+-----------+--------------+
|   Classify|Total Quantity|
+-----------+--------------+
|Overstocked|           320|
|     Normal|           415|
|  Low Stock|            95|
+-----------+--------------+



In [0]:

"""Classify employees for a bonus eligibility program. Employees in "Sales" and "Marketing" with
performance scores above 80 get a 20% bonus, while others with scores above 70 get 15%. All other
employees receive no bonus. Group by department and calculate total bonus allocation."""

employees = [
("karthik", "Sales", 85),
("neha", "Marketing", 78),
("priya", "IT", 90),
("mohan", "Finance", 65),
("ajay", "Sales", 55),
("vijay", "Marketing", 82),
("veer", "HR", 72),
("aatish", "Sales", 88),
("animesh", "Finance", 95),
("nishad", "IT", 60)
]
employees_df = spark.createDataFrame(employees, ["name", "department", "performance_score"])
base_salary = 100000

# Create DataFrame
employees_df = spark.createDataFrame(employees, ["name", "department", "performance_score"])

# Calculate bonus based on the eligibility criteria
employees_df = employees_df.withColumn(
    "bonus",
    when((col("department").isin("Sales", "Marketing")) & (col("performance_score") > 80), 0.20 * base_salary)
    .when((~col("department").isin("Sales", "Marketing")) & (col("performance_score") > 70), 0.15 * base_salary)
    .otherwise(0)
)

# Group by department and calculate total bonus allocation
bonus_allocation_df = employees_df.groupBy("department").sum("bonus").withColumnRenamed("sum(bonus)", "total_bonus")

# Show the result
bonus_allocation_df.show()
employees_df.show()

+----------+-----------+
|department|total_bonus|
+----------+-----------+
|     Sales|    40000.0|
| Marketing|    20000.0|
|        IT|    15000.0|
|   Finance|    15000.0|
|        HR|    15000.0|
+----------+-----------+

+-------+----------+-----------------+-------+
|   name|department|performance_score|  bonus|
+-------+----------+-----------------+-------+
|karthik|     Sales|               85|20000.0|
|   neha| Marketing|               78|    0.0|
|  priya|        IT|               90|15000.0|
|  mohan|   Finance|               65|    0.0|
|   ajay|     Sales|               55|    0.0|
|  vijay| Marketing|               82|20000.0|
|   veer|        HR|               72|15000.0|
| aatish|     Sales|               88|20000.0|
|animesh|   Finance|               95|15000.0|
| nishad|        IT|               60|    0.0|
+-------+----------+-----------------+-------+



In [0]:
"""
For each product, classify return reasons as "High Return Rate" if return count exceeds 100 and
satisfaction score below 50, "Moderate Return Rate" if return count is between 50-100 with a score
between 50-70, and "Low Return Rate" otherwise. Group by category to count product return rates"""


products = [
("Laptop", "Electronics", 120, 45),
("Smartphone", "Electronics", 80, 60),
("Tablet", "Electronics", 50, 72),
("Headphones", "Accessories", 110, 47),
("Shoes", "Clothing", 90, 55),
("Jacket", "Clothing", 30, 80),
("TV", "Electronics", 150, 40),
("Watch", "Accessories", 60, 65),
("Pants", "Clothing", 25, 75),
("Camera", "Electronics", 95, 58)
]
products_df = spark.createDataFrame(products, ["product_name", "category", "return_count",
"satisfaction_score"])

products_df = products_df.withColumn("classify",when((col("return_count")>100) & (col("satisfaction_score")<50),"High Return Rate")
                                     .when((col("return_count").between(50, 100)) & (col("satisfaction_score").between(50, 70)), "Moderate Return Rate")
                                     .otherwise("Low Return Rate"))

products_df_agg = products_df.groupby("classify","category").count().orderBy("category")
products_df_agg.show()
products_df.show()


+--------------------+-----------+-----+
|            classify|   category|count|
+--------------------+-----------+-----+
|    High Return Rate|Accessories|    1|
|Moderate Return Rate|Accessories|    1|
|Moderate Return Rate|   Clothing|    1|
|     Low Return Rate|   Clothing|    2|
|    High Return Rate|Electronics|    2|
|Moderate Return Rate|Electronics|    2|
|     Low Return Rate|Electronics|    1|
+--------------------+-----------+-----+

+------------+-----------+------------+------------------+--------------------+
|product_name|   category|return_count|satisfaction_score|            classify|
+------------+-----------+------------+------------------+--------------------+
|      Laptop|Electronics|         120|                45|    High Return Rate|
|  Smartphone|Electronics|          80|                60|Moderate Return Rate|
|      Tablet|Electronics|          50|                72|     Low Return Rate|
|  Headphones|Accessories|         110|                47|    High R

In [0]:
"""
Classify customers' spending as "High Spender" if spending exceeds $1000 with "Premium"
membership, "Average Spender" if spending between $500-$1000 and membership is "Standard",
and "Low Spender" otherwise. Group by membership and calculate average spending."""

customers = [
("karthik", "Premium", 1050, 32),
("neha", "Standard", 800, 28),
("priya", "Premium", 1200, 40),
("mohan", "Basic", 300, 35),
("ajay", "Standard", 700, 25),
("vijay", "Premium", 500, 45),
("veer", "Basic", 450, 33),
("aatish", "Standard", 600, 29),
("animesh", "Premium", 1500, 60),
("nishad", "Basic", 200, 21)
]
customers_df = spark.createDataFrame(customers, ["name", "membership", "spending", "age"])
customers_df = customers_df.withColumn("Classify",when((col("spending")>1000) & (col("membership")== "Premium") ,"High Spender")
                            .when((col("spending").between(500,1000)) & (col("membership")== "Standard"),"Average Spender")
                            .otherwise("Low Spender"))
customers_df_agg = customers_df.groupby("membership").avg("spending").withcolumnrenamed("avg(spending)", "AVG SPENDING")
customers_df_agg.show()
customers_df.show()

+----------+-----------------+
|membership|    avg(spending)|
+----------+-----------------+
|   Premium|           1062.5|
|  Standard|            700.0|
|     Basic|316.6666666666667|
+----------+-----------------+

+-------+----------+--------+---+---------------+
|   name|membership|spending|age|       Classify|
+-------+----------+--------+---+---------------+
|karthik|   Premium|    1050| 32|   High Spender|
|   neha|  Standard|     800| 28|Average Spender|
|  priya|   Premium|    1200| 40|   High Spender|
|  mohan|     Basic|     300| 35|    Low Spender|
|   ajay|  Standard|     700| 25|Average Spender|
|  vijay|   Premium|     500| 45|    Low Spender|
|   veer|     Basic|     450| 33|    Low Spender|
| aatish|  Standard|     600| 29|Average Spender|
|animesh|   Premium|    1500| 60|   High Spender|
| nishad|     Basic|     200| 21|    Low Spender|
+-------+----------+--------+---+---------------+



In [0]:
"""Classify orders as "Delayed" if delivery time exceeds 7 days and origin location is "International",
"On-Time" if between 3-7 days, and "Fast" if below 3 days. Group by product type to see the count of
each delivery speed category"""

orders = [
("Order1", "Laptop", "Domestic", 2),
("Order2", "Shoes", "International", 8),
("Order3", "Smartphone", "Domestic", 3),
("Order4", "Tablet", "International", 5),
("Order5", "Watch", "Domestic", 7),
("Order6", "Headphones", "International", 10),
("Order7", "Camera", "Domestic", 1),
("Order8", "Shoes", "International", 9),
("Order9", "Laptop", "Domestic", 6),
("Order10", "Tablet", "International", 4)
]

orders_df = spark.createDataFrame(orders, ["order_id", "product_type", "origin", "delivery_days"])
orders_df = orders_df.withColumn("Classify",when((col("delivery_days")>7) & (col("origin")=="International"),"Delayed")
                                 .when(col("delivery_days").between(3,7) ,"On-Time")
                                 .otherwise("FAST"))
orders_df_agg = orders_df.groupby("Classify","product_type").count()  
orders_df_agg.show()                               
orders_df.show()

+--------+------------+-----+
|Classify|product_type|count|
+--------+------------+-----+
|    FAST|      Laptop|    1|
| Delayed|       Shoes|    2|
| On-Time|  Smartphone|    1|
| On-Time|       Watch|    1|
| On-Time|      Tablet|    2|
| Delayed|  Headphones|    1|
|    FAST|      Camera|    1|
| On-Time|      Laptop|    1|
+--------+------------+-----+

+--------+------------+-------------+-------------+--------+
|order_id|product_type|       origin|delivery_days|Classify|
+--------+------------+-------------+-------------+--------+
|  Order1|      Laptop|     Domestic|            2|    FAST|
|  Order2|       Shoes|International|            8| Delayed|
|  Order3|  Smartphone|     Domestic|            3| On-Time|
|  Order4|      Tablet|International|            5| On-Time|
|  Order5|       Watch|     Domestic|            7| On-Time|
|  Order6|  Headphones|International|           10| Delayed|
|  Order7|      Camera|     Domestic|            1|    FAST|
|  Order8|       Shoes|Intern

In [0]:
"""Question Set:
1. Classify loan applicants as "High Risk" if the loan amount exceeds twice their income and
credit score is below 600, "Moderate Risk" if the loan amount is between 1-2 times their
income and credit score between 600-700, and "Low Risk" otherwise. Find the total count of
each risk level.
2. For applicants classified as "High Risk," calculate the average loan amount by income range
(e.g., < 50k, 50-100k, >100k).
3. Group by income brackets (<50k, 50-100k, >100k) and calculate the average credit score for
each risk level. Filter for groups where average credit score is below 650."""


loan_applicants = [
("karthik", 60000, 120000, 590),
("neha", 90000, 180000, 610),
("priya", 50000, 75000, 680),
("mohan", 120000, 240000, 560),
("ajay", 45000, 60000, 620),
("vijay", 100000, 100000, 700),
("veer", 30000, 90000, 580),
("aatish", 85000, 85000, 710),
("animesh", 50000, 100000, 650),
("nishad", 75000, 200000, 540)
]

loan_applicants_df = spark.createDataFrame(loan_applicants, ["name", "income", "loan_amount",
"credit_score"])
loan_applicants_df = loan_applicants_df.withColumn("Risk_Level",when((col("credit_score")<600) & (col("loan_amount")> 2*col("income")),"High Risk")
    .when((col("loan_amount") > col("income")) & (col("loan_amount") <= 2 * col("income")) & (col("credit_score").between(600, 700)), "Moderate Risk")
    .otherwise("Low Risk"))
loan_applicants_df = loan_applicants_df.withColumn(
    "Income_Range",
    when(col("income") < 50000, "<50k")
    .when((col("income") >= 50000) & (col("income") <= 100000), "50-100k")
    .otherwise(">100k")
)
loan_applicants_df_agg = loan_applicants_df.groupby("Risk_Level").count()
loan_applicants_df_agg.show()
high_risk_avg_loan_df = loan_applicants_df.filter(col("Risk_Level") == "High Risk") \
    .groupBy("Income_Range").agg(avg("loan_amount").alias("avg_loan_amount"))
high_risk_avg_loan_df.show()
loan_applicants_df.show()

avg_credit_score_df = loan_applicants_df.groupBy("Income_Range", "Risk_Level") \
    .agg(avg("credit_score").alias("avg_credit_score"))

# Filter for groups where average credit score is below 650
low_credit_score_df = avg_credit_score_df.filter(col("avg_credit_score") < 650)
low_credit_score_df.show()


+-------------+-----+
|   Risk_Level|count|
+-------------+-----+
|     Low Risk|    4|
|Moderate Risk|    4|
|    High Risk|    2|
+-------------+-----+

+------------+---------------+
|Income_Range|avg_loan_amount|
+------------+---------------+
|        <50k|        90000.0|
|     50-100k|       200000.0|
+------------+---------------+

+-------+------+-----------+------------+-------------+------------+
|   name|income|loan_amount|credit_score|   Risk_Level|Income_Range|
+-------+------+-----------+------------+-------------+------------+
|karthik| 60000|     120000|         590|     Low Risk|     50-100k|
|   neha| 90000|     180000|         610|Moderate Risk|     50-100k|
|  priya| 50000|      75000|         680|Moderate Risk|     50-100k|
|  mohan|120000|     240000|         560|     Low Risk|       >100k|
|   ajay| 45000|      60000|         620|Moderate Risk|        <50k|
|  vijay|100000|     100000|         700|     Low Risk|     50-100k|
|   veer| 30000|      90000|         

In [0]:
"""4. Categorize customers based on purchase recency: "Frequent" if last purchase within
30 days, "Occasional" if within 60 days, and "Rare" if over 60 days. Show the number of each
category per membership type.
5. Find the average total purchase amount for customers with "Frequent" purchase recency
and "Premium" membership.
6. For customers with "Rare" recency, calculate the minimum purchase amount across different
membership types"""

customer_purchases = [
("karthik", "Premium", 50, 5000),
("neha", "Standard", 10, 2000),
("priya", "Premium", 65, 8000),
("mohan", "Basic", 90, 1200),
("ajay", "Standard", 25, 3500),
("vijay", "Premium", 15, 7000),
("veer", "Basic", 75, 1500),
("aatish", "Standard", 45, 3000),
("animesh", "Premium", 20, 9000),
("nishad", "Basic", 80, 1100)
]
customer_purchases_df = spark.createDataFrame(customer_purchases, ["name", "membership",
"days_since_last_purchase", "total_purchase_amount"])
customer_purchases_df = customer_purchases_df.withColumn("Categorize",when(col("days_since_last_purchase")<30,"Frequent")
                                                         .when(col("days_since_last_purchase").between(30,60),"Rare")
                                                         .otherwise("Occasional"))
avg_total_purchase = customer_purchases_df.groupBy("Categorize", "membership") \
    .avg("total_purchase_amount") \
    .filter((col("Categorize") == "Frequent") & (col("membership") == "Premium")) 

min_purchase = customer_purchases_df.filter(col("Categorize") == "Rare") \
                                    .groupBy("membership") \
                                    .min("total_purchase_amount")

# Show the result
min_purchase.show()                                      
avg_total_purchase.show()
customer_purchases_df.show()

+----------+--------------------------+
|membership|min(total_purchase_amount)|
+----------+--------------------------+
|   Premium|                      5000|
|  Standard|                      3000|
+----------+--------------------------+

+----------+--------------------------+
|membership|min(total_purchase_amount)|
+----------+--------------------------+
|   Premium|                      5000|
|  Standard|                      3000|
+----------+--------------------------+

+----------+----------+--------------------------+
|Categorize|membership|avg(total_purchase_amount)|
+----------+----------+--------------------------+
|  Frequent|   Premium|                    8000.0|
+----------+----------+--------------------------+

+-------+----------+------------------------+---------------------+----------+
|   name|membership|days_since_last_purchase|total_purchase_amount|Categorize|
+-------+----------+------------------------+---------------------+----------+
|karthik|   Premium|     